In [1]:
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
vector = [1, 0.686, 0.00686]

a = torch.tensor(vector, dtype=torch.float32).to("cuda")
b = torch.tensor(vector, dtype=torch.float8_e4m3fn).to("cuda")
c = torch.tensor(vector, dtype=torch.float8_e5m2).to("cuda")

print(a)
print(b)
print(c)

tensor([1.0000, 0.6860, 0.0069], device='cuda:0')
tensor([1.0000, 0.6875, 0.0078], device='cuda:0', dtype=torch.float8_e4m3fn)
tensor([1.0000, 0.6250, 0.0068], device='cuda:0', dtype=torch.float8_e5m2)


In [3]:
a.to("cpu").to(torch.float8_e5m2)

tensor([1.0000, 0.6250, 0.0068], dtype=torch.float8_e5m2)

In [4]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, stride=2)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv2d(8, 8*8, 3, stride=2)
        self.relu2 = nn.ReLU()

        self.head = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(256, 128), 
            nn.Dropout(0.2),
            nn.Linear(128, 10) 
            # nn.Softmax()
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        y = self.head(x)
        return y

class ConvBN(torch.nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 3, stride=2)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)
    

class Head(torch.nn.Module):

    def __init__(self):
        super().__init__()

        self.module = nn.Sequential(
            nn.AvgPool2d(3, 2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256, 128), 
            nn.Dropout(),
            nn.Linear(128, 10)
            # nn.Softmax()
        )

    def forward(self, x):
        return self.module(x)

class ComposedClassifier(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.convbn1 = ConvBN(1, 8)
        self.convbn2 = ConvBN(8, 8*8)
        self.head = Head()

    def forward(self, x):
        x = self.convbn1(x)
        x = self.convbn2(x)
        y = self.head(x)
        return y
    

def train(dataloader, model, loss_fn, optimizer, dtype=None):
    size = len(dataloader.dataset)
    model.train()
    correct = 0
    for batch, (X, y) in enumerate(dataloader):
        if dtype is not None:
            X, y = X.to(dtype=dtype), y.to(dtype=dtype)
        X = X.to("cuda")
        y = y.to("cuda")
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            # print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            # print(correct)
            # print((len(y) * 100))
            # print(correct * 100. / (len(y) * 100))
            correct = 0


def test(dataloader, model, loss_fn, cuda=False, dtype=None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            if dtype is not None:
                X, y = X.to(dtype=dtype), y.to(dtype=dtype)
            if cuda:
                X, y = X.to('cuda'), y.to('cuda')
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
model = ComposedClassifier()

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

In [6]:
model = model.to("cuda")
model.eval()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

for i in range(20):
    train(train_dataloader, model, loss_fn, optimizer)


In [7]:
%%timeit
test(test_dataloader, model, loss_fn, cuda=True)

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

Test Error: 
 Accuracy: 73.7%, Avg loss: 0.755099 

248 ms ± 815 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
dtype = torch.float8_e4m3fn
fp8_model = model.to("cuda", dtype)
test(test_dataloader, fp8_model, loss_fn, cuda=True, dtype=dtype)

RuntimeError: getCudnnDataTypeFromScalarType() not supported for Float8_e4m3fn

In [15]:
print(fp8_model.type)

AttributeError: 'ComposedClassifier' object has no attribute 'dtype'